# Markov Chain Monte Carlo Lab

<h2>Using Markov Chain Monte Carlo to infer the parameters of a simple model
</h2>

In [ ]:
]add ForwardDiff

In [ ]:
]add LogExpFunctions

In [ ]:
using CairoMakie
using Optim
using ForwardDiff
using LogExpFunctions

In [ ]:
# Example data set from  arxiv:1008.4686, table 1 (https://arxiv.org/abs/1008.4686)
# You can also refer to that paper for more background, equations, etc.
alldata = [201. 592 61; 244 401 25; 47  583 38; 287 402 15; 203 495 21; 58  173 15; 210 479 27;
           202 504 14; 198 510 30; 158 416 16; 165 393 14; 201 442 25; 157 317 52; 131 311 16;
           166 400 34; 160 337 31; 186 423 42; 125 334 26; 218 533 16; 146 344 22 ]
# The first 5 data points are outliers; for the first part we'll just use the "good" data points
x    = alldata[6:end, 1]
y    = alldata[6:end, 2]
# this is the standard deviation (uncertainty) on the y measurements, also known as \sigma_i
yerr = alldata[6:end, 3];

In [ ]:
# To start, let's have a look at our data set.
f = Figure()
Axis(f[1, 1], xlabel="x", ylabel="y")
errorbars!(x, y, yerr);
scatter!(x, y, markersize=20);
f

For this lab, we are going to imagine that there is some physical reason to believe there is a *linear* relationship between the quantities $x$ and $y$, so our *generative model* of the process is that there is a "real" or "predicted" value $y_{\textrm{pred}}$ given by $y_{\textrm{pred}} = b + m x$, for some parameters $b$ and $m$ that we will try to infer.  Our *measurements* $y$ are noisy measurements of $y_{\textrm{pred}}$, with additive Gaussian noise with known variance, $\sigma_i$ for data point $i$.  ($\sigma$ is called `yerr` in this notebook.)  This is a strong assumption about our data-collection method.

With those assumptions, we can write down the *likelihood* for a single measurement $y_i$ given its corresponding $x_i$ and $\sigma_i$, and straight-line model parameters $b$ and $m$:

$y_{\textrm{pred},i} = b + m x_i$

  $p(y_i | m, b) = \frac{1}{\sqrt{2 \pi} \sigma_i} \exp(-\frac{(y - y_{\textrm{pred},i})^2}{2 \sigma_i^2})$


We are making a number of simplifying assumptions here:

* there are no uncertainties on the $x$ values
* there are additive Gaussian measurement uncertainties on the $y$ values with known standard deviations $\sigma$
* the data points are statistically independent

Since we are assuming the $x$ and $\sigma$ values are perfectly known, we will treat them as *constants* rather than *data* in the probability equations.

In practice, it is usually preferable to work in log-probabilities rather than
linear probabilities, because the probability values can be very small, and if we're not careful we can hit a numerical issue called *underflow*, where the numbers become so small that they can't be represented in standard floating-point numerical representation.

If the data points are statistically independent, then the likelihood of the whole collection of data points $y = \{ y_i \}$ is the *product* of their individual likelihoods:

$p(y | m,b) = \prod_i \frac{1}{\sqrt{2 \pi} \sigma_i} \exp(-\frac{(y - y_{\textrm{pred},i})^2}{2 \sigma_i^2})$

and taking the log,

$\log p(y | m,b) = \sum_i \log(\frac{1}{\sqrt{2 \pi} \sigma_i}) -\frac{(y - y_{\textrm{pred},i})^2}{2 \sigma_i^2}$

The first thing we will do is implement that log-likelihood function!


In [ ]:
function log_likelihood_one(params, x, y, yerr)
    """This function computes the log-likelihood of a data set with coordinates
    (x_i,y_i) and Gaussian uncertainties on y_i of yerr_i (aka sigma_i)

    The model is a straight line, so the model's predicted y values are
        y_pred_i = b + m x_i.

    params = (b,m) are the parameters (scalars)
    x,y,yerr are arrays (aka vectors)

    Return value is a scalar log-likelihood.
    """
    # unpack the parameters
    b,m = params
    # compute the vector y_pred, the model predictions for the y measurements
    y_pred = b .+ m .* x
    # compute the log-likelihoods for the individual data points
    # (the quantity inside the sum in the text above)
    ### FILL IN CODE HERE!  Implement the log-likelihood function from the text above!
    loglikes = #log.( .... ) -. 0.5 .* ().^ ./ ().^2
    # the log-likelihood for the whole vector of measurements is the sum of individual log-likelihoods
    loglike = sum(loglikes)
    return loglike
end;

<h2>Maximum likelihood</h2>

Before we start experimenting Markov Chain Monte Carlo, let's use a
stock optimizer routine from Julia's *Optim* package.  The optimizer will allow us to find the *maximum likelihood* paramaters $b$ and $m$.

Since the optimizer wants to *minimize*
a function but we want to *maximize* the log-likelihood, we need to add a
negative sign...

In [ ]:
# The optimizer we're using here requires an initial guess.  This log-likelihood happens
# to be pretty simple, so we don't need to work very hard to give it a good initial guess!
initial_params = [0., 0.]
# The "args" parameter here gets passed to the neg_ll_one function (after the parameters)
result = optimize(p -> -log_likelihood_one(p, x, y, yerr), initial_params)

In [ ]:
b_ml,m_ml = Optim.minimizer(result)

Previously, we used the jack-knife routine to estimate variances on our parameters.
We can also compute the second derivative of the log-likelihood function, at the peak,
to get an estimate of uncertainties on the parameters.  This is related to the Fisher
information matrix, if you've ever encountered that.

In [ ]:
# Don't worry about understanding this!
invhess = inv(ForwardDiff.hessian(p -> -log_likelihood_one(p, x, y, yerr), [b_ml,m_ml]))

In [ ]:
# The optimizer gives us the parameters that maximize the log-likelihood, along with an estimate of the uncertainties.
# To start, let's have a look at our data set.
f = Figure()
Axis(f[1, 1], xlabel="B", ylabel="M")
errorbars!(x, y, yerr);
scatter!(x, y, markersize=20);
xx = LinRange(50, 250, 50)
lines!(xx, b_ml .+ m_ml .* xx)
# Draw a sampling of B,M parameter values that are consistent with the fit,
# using the estimated inverse-Hessian matrix (parameter covariance)
using LinearAlgebra
# use the svd to draw multivariate random normal samples!
S = svd(invhess)
BM = [S.U * Diagonal(sqrt.(S.S)) * randn(2) for i in 1:10]
for (db,dm) in BM
    lines!(xx, (b_ml .+ db) .+ (m_ml .+ dm) .* xx, color=:cornflowerblue)
end
lines!(xx, b_ml .+ m_ml .* xx, color=:black, linewidth=5)
f

In [ ]:
# You can also plot the ellipse showing the constraints in B,M space by manipulating hess_inv
# (don't worry about understanding this math)
SS = S.U * Diagonal(sqrt.(S.S))
th = LinRange(0., 2π, 200)
xx = sin.(th)
yy = cos.(th)
dbm = SS * [xx yy]'
ellipse_b = b_ml .+ dbm[1,:]
ellipse_m = m_ml .+ dbm[2,:]

f = Figure()
Axis(f[1, 1], xlabel="B", ylabel="M", title="Parameter constraints from Maximum-Likelihood")
lines!(ellipse_b, ellipse_m, color=:red)
f

<h2>Markov Chain Monte Carlo</h2>

Next, let's implement the Markov Chain Monte Carlo algorithm.

The MCMC algorithm moves a "particle" or "sample" or "walker" randomly around the particle space, by first proposing a move, and then using the relative likelihoods of the current and proposed positions to decide whether to accept or reject the move.

In [ ]:
function mcmc(logprob_func,
              propose_func,
              initial_pos, nsteps)
    """
    MCMC: Markov Chain Monte Carlo.  Draw samples from the *logprob_func* probability distribution,
    using proposed moves generated by the function *propose_func*.

    * logprob_func: a function that returns the log-probability at a given value of parameters.
               It will get called like this:
        lnp = logprob_func(params, logprob_args)
    * propose_func: a function that proposes to jump to a new point in parameter space.
               It will get called like this:
        p_new = propose_func(p, propose_args)
    * initial_pos: initial position in parameter space (list/array)
    * nsteps: integer number of MCMC steps to take
    
    Returns  (chain, faccept)
    * chain: size Nsteps x P, MCMC samples
    * faccept: float: fraction of proposed jumps that were accepted
    """
    p = initial_pos
    logprob = logprob_func(p)
    chain = zeros(Float64, (nsteps, length(p)))
    naccept = 0
    for i in 1:nsteps
        # propose a new position in parameter space
        ### FILL IN CODE HERE -- propose a jump to a new place in param space
        p_new = #...
        # compute probability at new position
        ### FILL IN CODE HERE
        logprob_new = #...
        # decide whether to jump to the new position
        #### FILL IN CODE HERE!!!
        if exp( ..... ) > rand()
            p = p_new
            logprob = logprob_new
            naccept += 1
        end
        # save the position
        chain[i,:] = p
    end
    return chain, naccept/nsteps
end;

We'll use a Gaussian (without covariance between the parameters) for our proposal distribution:

In [ ]:
function propose_gaussian(p, stdevs)
    """
    A Gaussian proposal distribution for mcmc.
    *p*: the point in parameter space to jump from
    *stdevs*: standard deviations for each dimension in the parameter space.
    """
    return p .+ randn(length(p)) .* stdevs
end;

Now, we defined our log-likelihood function above, but when using MCMC for Bayesian inference, we need to pass it a log-*posterior* function.  That is,
we must include the log-prior for the parameters.  It is very common to see "uninformative" or "flat" priors used; in fact, it's not uncommon to see the log-prior just set to zero, as below, which is, statistically speaking, a naughty thing to do, since that prior definitely isn't a proper probability distribution -- it isn't even bounded!  But, it *feels* like we haven't imposed our *subjective* prior beliefs on the inference, which is why people often do it.  But you can't avoid subjectivity---if you change the parameterization, for example, a flat prior becomes non-flat!

In [ ]:
function log_posterior_one(params, x, y, err)
    loglike = log_likelihood_one(params, x, y, yerr)
    # Improper, flat priors on params!
    logprior = 0.
    return loglike + logprior
end;

In [ ]:
# initial B,M
initial_pos = [0., 1.0]
# proposal distribution: jump sizes for B,M
jump_sizes = [1., 0.1]
# Run MCMC!
chain,accept = mcmc(p -> log_posterior_one(p, x, y, err),
                    p -> propose_gaussian(p, jump_sizes),
                    initial_pos, 5000)
println("Fraction of moves accepted:", accept)
size(chain)

In [ ]:
# Plot the parameter values in the chain!
f = Figure()
Axis(f[1, 1], xlabel="B", ylabel="M", title="MCMC Samples")
scatter!(chain[:,1], chain[:,2], color=:grey)
lines!(ellipse_b, ellipse_m, color=:red)
f

Try re-running the MCMC cell above and re-plotting the results.  Do the results look the same every time?  Does that suggest anything to you about whether the chain has *converged* after the number of steps we have taken?

Try increasing the number of steps -- do the results look better?  How long are you willing to wait?

In [ ]:
# Plot the MCMC chains with respect to sample number
f = Figure(size=(1600, 500))
Axis(f[1, 1], ylabel="B", xlabel="MCMC Step")
lines!(chain[:,1])
Axis(f[1, 2], ylabel="M", xlabel="MCMC Step")
lines!(chain[:,2])
f

Looking at these plots of how the "particle" moves through the $B$,$M$ parameter space, what do you see?  How often does it traverse the whole space?  Do you think the step sizes are too big or too small?

In [ ]:
# Zoom in on the beginning of the chain to see the "burn-in", and repeated values
# Plot the MCMC chains with respect to sample number
f = Figure(size=(1600, 500))
Axis(f[1, 1], ylabel="B", xlabel="MCMC Step")
nstart=200
lines!(chain[1:nstart,1])
Axis(f[1, 2], ylabel="M", xlabel="MCMC Step")
lines!(chain[1:nstart,2])
f

In the plots above, observe that our MCMC algorithm is moving toward the "core" of the probability mass.  Also observe that there are horizontal segments in the plots --- where we have proposed a number of moves that have been rejected, so that there are repeated `B,M` values in our MCMC chain.


In [ ]:
size(chain,1)

In [ ]:
# Let's plot the models for a few randomly drawn MCMC samples.
# The optimizer gives us the parameters that maximize the log-likelihood, along with an estimate of the uncertainties.
# To start, let's have a look at our data set.
f = Figure()
Axis(f[1, 1], xlabel="X", ylabel="Y", title="Sampling of MCMC fits")
errorbars!(x, y, yerr);
scatter!(x, y, markersize=20, color=:black);
xx = LinRange(50, 250, 50)

# Drop this many samples for "burn-in"
nburn = 1000
for i in rand(nburn:size(chain,1), 10)
    # Draw some random entries from the chain
    b,m = chain[i,:]
    lines!(xx, b .+ m .* xx, color=:cornflowerblue)
end
f

Another thing we can do is histogram the $B$ and $M$ values.  In fact, let's make a "corner plot" where we also show the *marginal* distributions.  We'll use my little homebrewed `cornerplot` function.

In [ ]:
function cornerplot(x, names; resolution=(600,600))
    # how many columns of data
    dim = size(x, 2)
    # rows to plot
    idxs = 1:size(x,1)
    f = Figure(resolution=resolution)
    for i in 1:dim, j in 1:dim
        if i < j
            continue
        end
        ax = Axis(f[i, j], aspect = 1,
                  topspinevisible = false,
                  rightspinevisible = false,)
        if i == j
            hist!(x[idxs,i], direction=:y)
            ax.xlabel = names[i]
        else
            scatter!(x[idxs,j], x[idxs,i], markersize=4)
            ax.xlabel = names[j]
            ax.ylabel = names[i]
        end
    end
    f
end;

In [ ]:
cornerplot(chain, ["B","M"])

Try calling the MCMC routine again with step sizes of [0.1, 0.01]
and see what the plots look like!

<h3>Tuning MCMC proposal distribution step sizes</h3>

If we have two parameters, how do we know which step size we should adjust in
order to get a good acceptance ratio?

One approach is to modify our MCMC function so that instead of stepping in both
parameters at once, we alternate and step in only one parameter at each step of the algorithm.  We could try to write a fancy general version of that, but instead let's just copy-paste the MCMC routine and customize it for this task!  Once we've selected good step sizes we can go back to the regular version.

In [ ]:
function mcmc_cyclic(logprob_func,
                     propose_func,
                     initial_pos, nsteps)
    """
    This is a variation on the "vanilla" MCMC algorithm, where we change the proposal function
    to modify only a single parameter in each step of the MCMC.  We record the acceptance ratio
    separately for each parameter.
    """
    p = initial_pos
    logprob = logprob_func(p)
    chain = zeros(Float64, (nsteps, length(p)))
    naccept = zeros(Int, length(p))
    for i = 1:nsteps
        # propose a new position in parameter space
        p_jump = propose_func(p)
        # BUT, only copy one element (cycle through the elements);
        # keep the rest the same!
        # The index of the parameter to change:
        j = 1 + ((i-1) % length(p))
        p_new = copy(p)
        p_new[j] = p_jump[j]
        # compute probability at new position
        logprob_new = logprob_func(p_new)
        # decide whether to jump to the new position
        if exp(logprob_new - logprob) > rand()
            p = p_new
            logprob = logprob_new
            naccept[j] += 1
        end
        # save the position
        chain[i,:] = p
    end
    # Since we cycle through the parameters, the number of steps per parameter
    # is (approximately) (nsteps) / (number of parameters).
    return chain, naccept/(nsteps/length(p))
end;

In [ ]:
# initial B,M
initial_pos = [0., 1.0]
# proposal distribution: jump sizes for B,M
##### CHANGE THESE -- we were using [1, 0.1] before.  Play around with these values until you get
##### acceptance ratios of about 0.5 per coordinate!
jump_sizes = [1., 0.1]
# Run MCMC!
chain,accept = mcmc_cyclic(
    p -> log_posterior_one(p, x, y, yerr),
    p -> propose_gaussian(p, jump_sizes),
    initial_pos, 5000)
println("Fraction of moves accepted (for B & M, respective):", accept)
size(chain)

Try modifying the `jump_sizes` values above until you get acceptance ratios of about 0.5 for each parameter.  Recall that proposing smaller jumps should result in a larger acceptance ratio.


In [ ]:
# Now let's plug those *jump_sizes* into our "vanilla" MCMC.  Since we are now jumping in both parameters
# at once, the acceptance ratio will be a bit smaller.
# initial B,M
initial_pos = [0., 1.0]
# proposal distribution: jump sizes for B,M
jump_sizes #=  FROM ABOVE
# Run MCMC!
chain,accept = mcmc(log_posterior_one, (x,y,yerr),
                    propose_gaussian, jump_sizes,
                    initial_pos, 5000)
println("Fraction of moves accepted:", accept)
size(chain)

In [ ]:
# Let's have a look at the resulting samples!
f = Figure(size=(1600, 500))
Axis(f[1, 1], ylabel="B", xlabel="MCMC Step")
lines!(chain[:,1])
Axis(f[1, 2], ylabel="M", xlabel="MCMC Step")
lines!(chain[:,2])
Axis(f[1, 3], xlabel="B", ylabel="M")
scatter!(chain[:,1], chain[:,2])
# Also plot up the elliptical constraint we got from the optimizer.
lines!(ellipse_b, ellipse_m, color=:red, linewidth=5)
f

That looks way better!  Our samples are traversing the state space many times.

Let's also look at the resulting corner plot.

In [ ]:
nburn = 1000
cornerplot(chain, ["B","M"])

<h3>Extensions</h3>

* <b>If you are interested, try extending the model from a linear model to a quadratic model.  That is, switch to $y_{\textrm{pred}} = b + m x + q x^2$.  You will need to write a new `log_likelihood_quadratic` function that expects three parameters.  Run MCMC on that model, plot the results, and show the corner plots.  Does it look like the model "needs" the quadratic term?</b>

* <b>What if the $\sigma$ values are estimated incorrectly?  (Eg, if your experimenter friends overlooked or mis-estimated some source of error in their data collection!)  Try increasing or decreasing the `yerr` values by a factor of 2 and re-make the plots.  How do the constraints on $B$ and $M$ change?  How does the visual quality of the fit change?

* <b>We found jump sizes for $B$ and $M$ the led to okay acceptance ratios, but we are still taking jumps independently in the two variables, while we can clearly see that the variables are correlated.  Can you come up with a new `propose_func` that proposes jumps drawn from a Gaussian with appropriate covariance?  (you can check out where I sample from the inverse-Hessian ellipse, above, for how to sample from a multivariate Gaussian distribution given its covariance).

* <b>We used "uninformative" priors on $B$ and $M$.  Try changing that -- for example, try placing a Gaussian (log-)prior on one of the parameters, and see how that affects your samplings.  How strong do you have to make the prior for it to have a significant effect on your results?

I hope this has been an interesting glimpse into Markov Chain Monte Carlo in practice!